In [1]:
from transformers import pipeline
import json
import pandas as pd
import tqdm

In [2]:
from sentence_transformers import SentenceTransformer, util

In [3]:
with open('/content/test-A%2FSPIQA_testA.json', 'r') as file:
    data = json.load(file)

rows = []
for paper, content in data.items():
    for qa in content['qa']:
        rows.append({
            'paper': paper,
            'question': qa['question'],
            'answer': qa['answer'],
            'reference_figure': qa['reference'],
            'reference_figure_caption': content['all_figures'][qa['reference']]['caption']
        })

In [11]:
with open('/content/test-A%2FSPIQA_testA.json', 'r') as file:
    data = json.load(file)

In [4]:
df = pd.DataFrame(rows)
df.head()

,paper,question,answer,reference_figure,reference_figure_caption
0,1611.04684v1,What are the main differences between the educ...,The Bonaparte school focuses on outdoor physic...,1611.04684v1-Table1-1.png,A difficult example from QA
1,1611.04684v1,Which model performs the best for response sel...,The KEHNN model performs the best for response...,1611.04684v1-Table4-1.png,Table 4: Evaluation results on response selection
2,1611.04684v1,Which model performs best on the Ubuntu datase...,KEHNN,1611.04684v1-Table5-1.png,Accuracy on different length of text
3,1611.04684v1,What is the role of the knowledge gates in the...,The knowledge gates are responsible for select...,1611.04684v1-Figure1-1.png,Architecture of KEHNN
4,1611.04684v1,How does the average number of answers per que...,The training set has a higher average number o...,1611.04684v1-Table2-1.png,Table 2: Statistics of the answer selection da...


In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
question_embeddings = model.encode(df['question'].tolist(), convert_to_tensor=True)

In [8]:
predicted_answers = []
for idx, question in enumerate(df['question']):
    query_embedding = question_embeddings[idx]

    cosine_scores = util.cos_sim(query_embedding, question_embeddings).squeeze(0)
    cosine_scores[idx] = -1
    nn_index = int(cosine_scores.argmax())

    predicted_answers.append(df.loc[nn_index, 'answer'])

In [9]:
df['generated_answer'] = predicted_answers

In [10]:
df.to_csv('sentence_bert_results.csv', index=False)